In [5]:
from os import listdir
import string
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

doc = load_doc("/home/ifte/amiebot_project/Large_files/cnn_part_dataset/fffcaffda91f80b841efaefe04704e0357e4276c.story")

doc

"Kabul , Afghanistan -LRB- CNN -RRB- -- Military divers have found the body of a U.S. paratrooper who went missing last week in a river in western Afghanistan .\n\nThe NATO-led International Security Assistance Force said the soldier was found close to where he disappeared November 4 . Officials are continuing their search for a second paratrooper lost at the same time .\n\nBoth men -- from the 4th Brigade Combat Team , 82nd Airborne Division -- disappeared in the Morghab River near Afghanistan 's border with Turkmenistan . The men were on a routine resupply mission , the NATO group said .\n\nFamily members identified the recovered body as that of Benjamin Sherman , 21 , of Plymouth , Massachusetts .\n\nThey said he jumped into the river when he saw a fellow soldier struggling in the water .\n\n`` I know that day he jumped into the river to try to save his comrade was because he did n't just see another soldier in the water ; he saw his brother , '' said Sherman 's sister , Meredith , 

In [9]:
a=[10000, 5000, 15000, 2000]
   

import pandas as pd
d = pd.DataFrame()
d['a'] = a
d.head()

,a
0,10000
1,5000
2,15000
3,2000


In [6]:
# split a document into news story and highlights
def split_story(doc):
	# find first highlight
	index = doc.find('@highlight')
	# split into story and highlights
	story, highlights = doc[:index], doc[index:].split('@highlight')
	# strip extra white space around each highlight
	highlights = [h.strip() for h in highlights if len(h) > 0]
	return story, highlights

story, highlights = split_story(doc)
story, highlights

("Kabul , Afghanistan -LRB- CNN -RRB- -- Military divers have found the body of a U.S. paratrooper who went missing last week in a river in western Afghanistan .\n\nThe NATO-led International Security Assistance Force said the soldier was found close to where he disappeared November 4 . Officials are continuing their search for a second paratrooper lost at the same time .\n\nBoth men -- from the 4th Brigade Combat Team , 82nd Airborne Division -- disappeared in the Morghab River near Afghanistan 's border with Turkmenistan . The men were on a routine resupply mission , the NATO group said .\n\nFamily members identified the recovered body as that of Benjamin Sherman , 21 , of Plymouth , Massachusetts .\n\nThey said he jumped into the river when he saw a fellow soldier struggling in the water .\n\n`` I know that day he jumped into the river to try to save his comrade was because he did n't just see another soldier in the water ; he saw his brother , '' said Sherman 's sister , Meredith ,

In [7]:
# load all stories in a directory
def load_stories(directory):
	stories = list()
	for name in listdir(directory):
		filename = directory + '/' + name
		# load document
		doc = load_doc(filename)
		# split into story and highlights
		story, highlights = split_story(doc)
		# store
		stories.append({'story':story, 'highlights':highlights})
	return stories

# load stories
directory = '/home/ifte/amiebot_project/Large_files/cnn_part_dataset/'
stories = load_stories(directory)
print('Loaded Stories %d' % len(stories))


Loaded Stories 30


In [8]:
# clean a list of lines
def clean_lines(lines):
	cleaned = list()
	# prepare a translation table to remove punctuation
	table = str.maketrans('', '', string.punctuation)
	for line in lines:
		# strip source cnn office if it exists
		#index = line.find('(CNN) -- ')
		#if index > -1:
		#	line = line[index+len('(CNN)'):]
		# tokenize on white space
		line = line.split()
		# convert to lower case
		line = [word.lower() for word in line]
		# remove punctuation from each token
		line = [w.translate(table) for w in line]
		# remove tokens with numbers in them
		line = [word for word in line if word.isalpha()]
		# store as string
		cleaned.append(' '.join(line))
	# remove empty strings
	cleaned = [c for c in cleaned if len(c) > 0]
	return cleaned
 

for example in stories:
	example['story'] = clean_lines(example['story'].split('\n'))
	example['highlights'] = clean_lines(example['highlights'])

In [ ]:
example['story']

In [9]:
example['highlights']

['the tony nominations have been announced',
 'kinky boots gained the most with noms',
 'tony awards will be broadcast on cbs on june']

In [10]:
from bert_juman import JumanTokenizer
juman_tokenizer = JumanTokenizer()

from pytorch_pretrained_bert import BertTokenizer
mod_txt = "/home/ifte/Downloads/L12_H768_A12_E30_BPE_WWM_Transformers-Model/vocab.txt"
bert_tokenizer = BertTokenizer(mod_txt, do_lower_case=False, do_basic_tokenize=False)

In [14]:
''.join(example['highlights'])

'the tony nominations have been announcedkinky boots gained the most with nomstony awards will be broadcast on cbs on june'

In [19]:
#tokens = juman_tokenizer.tokenize(''.join(example['highlights']))
#bert_tokens = bert_tokenizer.tokenize(" ".join(tokens))
bert_tokens = bert_tokenizer.tokenize(''.join(example['highlights']))

In [25]:
for line in example['highlights']:
    print(bert_tokenizer.tokenize(line))

['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]']
['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]']
['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]']


In [2]:
def encode_for_summarization(story_lines, summary_lines, tokenizer, juman_tokenizer):
    """ Encode the story and summary lines, and join them
    as specified in [1] by using `[SEP] [CLS]` tokens to separate
    sentences.
    """
    
    tokens = juman_tokenizer.tokenize(story_lines)
    bert_tokens = bert_tokenizer.tokenize(" ".join(tokens))
    
    
    story_lines_token_ids = [
        bert_tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
        for line in story_lines
    ]
    
    tokens = juman_tokenizer.tokenize(summary_lines)
    bert_tokens = bert_tokenizer.tokenize(" ".join(tokens))
    
    summary_lines_token_ids = [
        #tokenizer.add_special_tokens_single_sequence(tokenizer.encode(line))
        bert_tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
        for line in summary_lines
    ]

    story_token_ids = [
        token for sentence in story_lines_token_ids for token in sentence
    ]
    summary_token_ids = [
        token for sentence in summary_lines_token_ids for token in sentence
    ]

    return story_token_ids, summary_token_ids

from bert_juman import JumanTokenizer
juman_tokenizer = JumanTokenizer()

from pytorch_pretrained_bert import BertTokenizer
mod_txt = "/home/ifte/Downloads/L12_H768_A12_E30_BPE_WWM_Transformers-Model/vocab.txt"
bert_tokenizer = BertTokenizer(mod_txt, do_lower_case=False, do_basic_tokenize=False)

encode_for_summarization(example['story'], example['highlights'], bert_tokenizer, juman_tokenizer)